### CONFIGURATIONS

Lets start by making some configurations

In [1]:
# Kafka Broker infos
kafkaBootstrapServers = "localhost:9092" 
kafkaUser = "MY_USER"
kafkaSecret = "MY_SECRET"

# Schema registry infos
schemaRegistryApiKey = "MY_API_KEY"
schemaRegistrySecret = "MY_SECRET"
schemaRegistryUrl = "..."

# Topic to consume
topicName = "gps"